In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from layers import *

In [3]:
X = np.random.normal(size=[128, 32, 32, 16])

In [4]:
inputs = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 16])
random_rolls = tf.placeholder(dtype=tf.float32, shape=[54])

In [5]:
def architecture(inputs, random_rolls, training):
    P = 0.5
    L = 54
    l = 1

    out = first_layer(inputs, training, 'input')

    with tf.variable_scope('stack1', reuse=tf.AUTO_REUSE):
        for i in range(1, 19):
            p = 1 - l/L * (1 - P)
            out = residual_block(out, 16, p, random_rolls[l-1], training, 'res'+str(i))
            l += 1

    with tf.variable_scope('stack2', reuse=tf.AUTO_REUSE):
        p = 1 - l/L * (1 - P)
        out = transition_block(out, 32, p, random_rolls[l-1], training, 'res'+str(1))
        l += 1

        for i in range(2, 19):
            p = 1 - l/L * (1 - P)
            out = residual_block(out, 32, p, random_rolls[l-1], training, 'res'+str(i))
            l += 1

    with tf.variable_scope('stack3', reuse=tf.AUTO_REUSE):
        p = 1 - l/L * (1 - P)
        out = transition_block(out, 64, p, random_rolls[l-1], training, 'res'+str(1))
        l += 1

        for i in range(2, 19):
            p = 1 - l/L * (1 - P)
            out = residual_block(out, 64, p, random_rolls[l-1], training, 'res'+str(i))

    return output_layer(out, 'out', 10)

# Use ADAM?
# Use better initializations?
# give probs as feed dict

In [6]:
# weights = [var for var in tf.trainable_variables() if 'weights' in var.name]
# lr = 0.1
# weight_decay = 1e-4
# momentum = 0.9
# out = architecture(inputs, survives, 'training')
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out,
#                                                               labels=labels)) + 
#        weight_decay * tf.add_n([tf.nn.l2_loss(weight) for weight in weights])
# step = tf.train.MomentumOptimizer(lr, momentum, use_nesterov=True).minimize(loss)
# with tf.Session() as sess: 
#     sess.run(tf.global_variables_initializer())
#     for epoch in range(500):
#         if epoch in (250, 375):
#             lr /= 10
#         for minibatch in minibatches:
#             random_rolls_batch = np.random.uniform(size=54)
#             sess.run(step, feed_dict={inputs: X_train_batch,
#                                       random_rolls: random_rolls_batch,
#                                       labels: y_train_labels})


In [7]:
# confirm tensorboard
random_rolls_batch = np.random.uniform(size=54)
out = architecture(inputs, random_rolls, True)
with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(tf.global_variables_initializer())
    sess.run(out, feed_dict={inputs: X,
                             random_rolls: random_rolls_batch})
    writer.close()

In [8]:
# # confirm training = False gives same answers
# with tf.Session() as sess:
#     ls = []
#     with tf.variable_scope('stoch_depth', reuse=tf.AUTO_REUSE):
#         for i in range(3):
#             out = architecture(inputs, False)
#             if i == 0:
#                 sess.run(tf.global_variables_initializer())
#             ls.append(sess.run(out, feed_dict={inputs: X}))

In [9]:
# # confirm training = False but new init gives diff answers
# with tf.Session() as sess:
#     ls = []
#     with tf.variable_scope('stoch_depth', reuse=tf.AUTO_REUSE):
#         for i in range(3):
#             out = architecture(inputs, False)
#             sess.run(tf.global_variables_initializer())
#             ls.append(sess.run(out, feed_dict={inputs: X}))

In [10]:
# confirm training = True gives diff answers
# with tf.Session() as sess:
#     ls = []
#     with tf.variable_scope('stoch_depth', reuse=tf.AUTO_REUSE):
#         for i in range(3):
#             out = architecture(inputs, True)
#             if i == 0:
#                 sess.run(tf.global_variables_initializer())
#             ls.append(sess.run(out, feed_dict={inputs: X}))